In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [2]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
# from openpyxl_image_loader import SheetImageLoader

In [3]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [8]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [9]:
def get_sign(field, sign_field, occ, value):
    if sign_field == 'Cor':
        value=value.capitalize()
    elif sign_field == 'Tipo de Película':
        value=value.split(' ')[1]
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [10]:
!cat .env

STAGE=CCR_PRODUCTION


In [14]:
# filename = 'acessos_importacao_v2'
filename='Sinalização H. Longitudinal BR 116 - Complemento' # 16/05/23

In [15]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]
ws

<Worksheet "MFA3 Long BR 116 RJ SP">

In [16]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
#         for i, img in enumerate(obj['img']):
#             if i == 0:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 1_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 1_1']
#             elif i == 1:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 1_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 1_2']
#             elif i == 2:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 2_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 2_1']
#             elif i == 3:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 2_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 2_2']
#             elif i == 4:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 3_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 3_1']
#             elif i == 5:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 3_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 3_2']
#             elif i == 6:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 4_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 4_1']
#             elif i == 7:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 4_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 4_2']

        
    values.append(obj)

In [17]:
values[0]

{'Identificação ANTT': 'SH 116 SP 140+000 N 1',
 'km': 140,
 'km final': 130,
 'Status': 'Identificado',
 'Equipe': 'Civil: SGS',
 'Encontrado em': datetime.datetime(2022, 10, 15, 0, 0),
 'Executado em': datetime.datetime(2022, 10, 15, 0, 0),
 'Sentido': 'Norte',
 'Classe': 'Sinalização horizontal (Longitudinal)',
 'Faixa': 'Expressa',
 'Rodovia': 'BR-116 SP',
 'Segmento': 'Trecho do Km inicial 140 ao Km final 130',
 'Observações': None,
 'Habilitar estação 1': 'SIM',
 'km da estação 1': 139,
 'lat. estação 1': -23.164423,
 'long. estação 1': -45.809754,
 'Faixa de sinalização 1_1': 'BORDO DIREITO',
 'Cor estação 1_1': 'BRANCA',
 'Valor mínimo 1_1': 100,
 'Faixa de sinalização 1_2': 'EIXO',
 'Cor estação 1_2': 'BRANCA',
 'Valor mínimo 1_2': 100,
 'Faixa de sinalização 1_3': 'BORDO ESQUERDO',
 'Cor estação 1_3': 'BRANCA',
 'Valor mínimo 1_3': 100,
 'Faixa de sinalização 1_4': None,
 'Cor estação 1_4': None,
 'Valor mínimo 1_4': None,
 'Faixa de sinalização 1_5': None,
 'Cor estação 1_5'

In [18]:
company = Company.objects.get(name='CCR - Rio Sp')

occurrence_type=OccurrenceType.objects.get(uuid='58b2dc08-7fca-4408-940c-22deca1a9af4')
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <OccurrenceType: Sinalização horizontal (Longitudinal) - ['CCR - Rio Sp']>)

In [19]:
for a in tqdm(values):
    a['Estação um'] = list(filter(None, [{'sinalization_lane': get_sign("Estação 1","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 1_1"]), 'station_color': get_sign('Estação 1', 'Cor estação 1', occurrence_type, a['Cor estação 1_1']),'minimal_value_one':a['Valor mínimo 1_1']} if a["Faixa de sinalização 1_1"] is not None else None,
                                        {'sinalization_lane': get_sign("Estação 1","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 1_2"]), 'station_color': get_sign('Estação 1', 'Cor estação 1', occurrence_type, a['Cor estação 1_2']),'minimal_value_one':a['Valor mínimo 1_2']} if a["Faixa de sinalização 1_2"] is not None else None,
                                        {'sinalization_lane': get_sign("Estação 1","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 1_3"]), 'station_color': get_sign('Estação 1', 'Cor estação 1', occurrence_type, a['Cor estação 1_3']),'minimal_value_one':a['Valor mínimo 1_3']} if a.get('Faixa de sinalização 1_3') and a["Faixa de sinalização 1_3"] is not None else None,
                                        {'sinalization_lane': get_sign("Estação 1","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 1_4"]), 'station_color': get_sign('Estação 1', 'Cor estação 1', occurrence_type, a['Cor estação 1_4']),'minimal_value_one':a['Valor mínimo 1_4']} if a.get('Faixa de sinalização 1_4') and a["Faixa de sinalização 1_4"] is not None else None,
                                        {'sinalization_lane': get_sign("Estação 1","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 1_5"]), 'station_color': get_sign('Estação 1', 'Cor estação 1', occurrence_type, a['Cor estação 1_5']),'minimal_value_one':a['Valor mínimo 1_5']} if a.get('Faixa de sinalização 1_5') and a["Faixa de sinalização 1_5"] is not None else None]))

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
for a in tqdm(values):
    a['Estação dois'] = list(filter(None, [{'sinalization_lane_two': get_sign("Estação 2","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 2_1"]), 'station_color_two': get_sign('Estação 2', 'Cor estação 2', occurrence_type, a['Cor estação 2_1']),'minimal_value_one_two':a['Valor mínimo 2_1']} if a["Faixa de sinalização 2_1"] is not None else None,
                                        {'sinalization_lane_two': get_sign("Estação 2","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 2_2"]), 'station_color_two': get_sign('Estação 2', 'Cor estação 2', occurrence_type, a['Cor estação 2_2']),'minimal_value_one_two':a['Valor mínimo 2_2']} if a["Faixa de sinalização 2_2"] is not None else None,
                                        {'sinalization_lane_two': get_sign("Estação 2","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 2_3"]), 'station_color_two': get_sign('Estação 2', 'Cor estação 2', occurrence_type, a['Cor estação 2_3']),'minimal_value_one_two':a['Valor mínimo 2_3']} if a.get('Faixa de sinalização 2_3') and a["Faixa de sinalização 2_3"] is not None else None,
                                        {'sinalization_lane_two': get_sign("Estação 2","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 2_4"]), 'station_color_two': get_sign('Estação 2', 'Cor estação 2', occurrence_type, a['Cor estação 2_4']),'minimal_value_one_two':a['Valor mínimo 2_4']} if a.get('Faixa de sinalização 2_4') and a["Faixa de sinalização 2_4"] is not None else None,
                                          {'sinalization_lane_two': get_sign("Estação 2","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 2_5"]), 'station_color_two': get_sign('Estação 2', 'Cor estação 2', occurrence_type, a['Cor estação 2_5']),'minimal_value_one_two':a['Valor mínimo 2_5']} if a.get('Faixa de sinalização 2_5') and a["Faixa de sinalização 2_5"] is not None else None]))

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
for a in tqdm(values):
    a['Estação tres'] = list(filter(None, [{'sinalization_lane_three': get_sign("Estação 3","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 3_1"]), 'station_color_three': get_sign('Estação 3', 'Cor estação 3', occurrence_type, a['Cor estação 3_1']),'minimal_value_one_three':a['Valor mínimo 3_1']} if a["Faixa de sinalização 3_1"] is not None else None,
                                        {'sinalization_lane_three': get_sign("Estação 3","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 3_2"]), 'station_color_three': get_sign('Estação 3', 'Cor estação 3', occurrence_type, a['Cor estação 3_2']),'minimal_value_one_three':a['Valor mínimo 3_2']} if a["Faixa de sinalização 3_2"] is not None else None,
                                        {'sinalization_lane_three': get_sign("Estação 3","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 3_3"]), 'station_color_three': get_sign('Estação 3', 'Cor estação 3', occurrence_type, a['Cor estação 3_3']),'minimal_value_one_three':a['Valor mínimo 3_3']} if a.get('Faixa de sinalização 3_3') and a["Faixa de sinalização 3_3"] is not None else None,
                                        {'sinalization_lane_three': get_sign("Estação 3","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 3_4"]), 'station_color_three': get_sign('Estação 3', 'Cor estação 3', occurrence_type, a['Cor estação 3_4']),'minimal_value_one_three':a['Valor mínimo 3_4']} if a.get('Faixa de sinalização 3_4') and a["Faixa de sinalização 3_4"] is not None else None,
                                          {'sinalization_lane_three': get_sign("Estação 3","Faixa de sinalização",occurrence_type,a["Faixa de sinalização 3_5"]), 'station_color_three': get_sign('Estação 3', 'Cor estação 3', occurrence_type, a['Cor estação 3_5']),'minimal_value_one_three':a['Valor mínimo 3_5']} if a.get('Faixa de sinalização 3_5') and a["Faixa de sinalização 3_5"] is not None else None]))

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
values[2]

IndexError: list index out of range

In [23]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], float(str(a['km']).replace('+','.')), str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road.name != a['Rodovia']:
        print(road.name,a['Rodovia'],)
        
    try:
        objects.append(Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa']),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=float(str(a['km']).replace('+','.')),
                    end_km_manually_specified= True if 'km final' in a else False,
                    end_km=float(a['km final']) if 'km final' in a and a['km final'] is not None else None,
                    point=point,
                    form_data={
                        'index': a['Índice (Serial Antt)'] if a.get('Índice (Serial Antt)') else None,
                        "id_ccr_antt": a['Identificação ANTT'] if a.get('Identificação ANTT') else None,
                        'segment': a['Segmento'],
                        "rame": a['Ramo'] if 'Ramo' in a else None,
                        'enable_station_one': True if a['Faixa de sinalização 1_1'] is not None else False,
                        'km_station': a['km da estação 1'] if a['Faixa de sinalização 1_1'] is not None else None,
                        'lat_station': a['lat. estação 1'] if a['Faixa de sinalização 1_1'] is not None else None,
                        'long_station': a['long. estação 1'] if a['Faixa de sinalização 1_1'] is not None else None,
                        'station_one': a['Estação um'],
                        'enable_station_two': True if a.get('Faixa de sinalização 2_1') and a['Faixa de sinalização 2_1'] is not None else False,
                        'km_station_two': a['km da estação 2'] if a.get('Faixa de sinalização 2_1') and a['Faixa de sinalização 2_1'] is not None else None,
                        'lat_station_two': a['lat. estação 2'] if a.get('Faixa de sinalização 2_1') and a['Faixa de sinalização 2_1'] is not None else None,
                        'long_station_two': a['long. estação 2'] if a.get('Faixa de sinalização 2_1') and a['Faixa de sinalização 2_1'] is not None else None,
                        'station_two': a['Estação dois'] if a.get('Estação dois') else None,
                        'enable_station_three': True if a.get('Faixa de sinalização 3_1') and a['Faixa de sinalização 3_1'] is not None else False,
                        'km_station_three': a['km da estação 3'] if a.get('Faixa de sinalização 3_1') and a['Faixa de sinalização 3_1'] is not None else None,
                        'lat_station_three': a['lat. estação 3'] if a.get('Faixa de sinalização 3_1') and a['Faixa de sinalização 3_1'] is not None else None,
                        'long_station_three': a['long. estação 3'] if a.get('Faixa de sinalização 3_1') and a['Faixa de sinalização 3_1'] is not None else None,
                        'station_three': a['Estação tres'] if a.get('Estação tres') else None,
                        'average': a['Valor Mínimo'] if a.get('Valor Mínimo') else None,
                        'notes': a['Observações'] if 'Observações' in a else None,
                        },
                    ))
    except Exception as e:
        print(f'AQUI: km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2}{e}')
        pass

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
objects[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f9e2f1f0190>,
 'uuid': UUID('f6355c94-f131-4cb7-b290-4f50262620fb'),
 'number': '',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 SP',
 'road_id': 1,
 'km': 140.0,
 'end_km': 130.0,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f9e30c972e0>,
 'direction': '0',
 'lane': '29',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('58b2dc08-7fca-4408-940c-22deca1a9af4'),
 'form_data': {'index': None,
  'id_ccr_antt': 'SH 116 SP 140+000 N 1',
  'segment': 'Trecho do Km inicial 140 ao Km final 130',
  'rame': None,
  'enable_station_one': True,
  'km_station': 139,
  'lat_station': -23.164423,
  'long_station': -45.809754,
  'station_one': [{'sinalization_lane': '1',
    'station_color': '1',
    'minimal_value_one': 100},
   {'sinalization_lane': '2',

In [25]:
for reporting in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/1 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'


/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2022-10-15 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-10-15 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2022-10-15 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: Da

Searching KM on direction...
Searching KM without direction...
Found KM on Road 1: BR-116 SP
CCR-Inv-2023.63750


In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]

In [4]:
for a in Reporting.objects.filter(company__uuid='3af64f25-59e8-446b-bb1e-963549090b0d',
                              occurrence_type__uuid='58b2dc08-7fca-4408-940c-22deca1a9af4'):
    if '-' not in str(a.form_data.get('lat_station_two')) and a.form_data.get('lat_station_two'):
        a.form_data['lat_station_two'] = float('-'+str(a.form_data['lat_station_two']))
    if '-' not in str(a.form_data.get('long_station_two')) and a.form_data.get('long_station_two'):
        a.form_data['long_station_two'] = float('-'+str(a.form_data['long_station_two']))
    
    if '-' not in str(a.form_data.get('lat_station_three')) and a.form_data.get('lat_station_three'):
        a.form_data['lat_station_three'] = float('-'+str(a.form_data['lat_station_three']))
    if '-' not in str(a.form_data.get('long_station_three')) and a.form_data.get('long_station_three'):
        a.form_data['long_station_three'] = float('-'+str(a.form_data['long_station_three']))
    a.save()

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2036: BR-101 SP
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2040: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2041: BR-101 RJ
Searching KM on direction...
Searching K

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
Searchin

In [3]:
a.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fd49b129490>,
 'uuid': UUID('674d3758-3e46-4c53-94b9-d877f09e8d6d'),
 'number': 'CCR-Inv-2023.63718',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-101 RJ',
 'road_id': 2040,
 'km': 380.8,
 'end_km': 390.0,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.0,
 'point': <Point object at 0x7fd49b120370>,
 'direction': '1',
 'lane': '29',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('58b2dc08-7fca-4408-940c-22deca1a9af4'),
 'form_data': {'rame': None,
  'index': None,
  'notes': None,
  'average': None,
  'segment': 'Trecho do Km incial 380,8 ao  Km final 390',
  'km_station': 382,
  'id_ccr_antt': 'SH 101 RJ 380+800 S 1',
  'lat_station': -22.894396,
  'station_one': [{'station_color': '1',
    'minimal_value_one': 100,
    'sinalization_lane': '1'},
   {'station_color': '1', 'mi